## Why TEEHR?
### Framework for standardized data and tooling for exploratory hydrologic evaluations at scale.
```
Tools for Exploratory Evaluation in Hydrologic Research TEEHR (pronounced "tier") is an opinionated Python tool set for loading, storing, and processing, high-volume hydrologic simulation data for the purpose of exploring and evaluating the datasets to assess their skill and performance.
```
---

#### TEEHR Documentation
https://rtiinternational.github.io/teehr/getting_started/index.html#getting-started

---

In [ ]:
from pathlib import Path
import os

import teehr
import pandas as pd

from utils import teehr_utils

# Enable use of accessor
from bokeh.io import output_notebook
output_notebook()

In [ ]:
NGIAB_OUTPUT_DIR = Path("/app/data")

TEEHR_EVALUATION_DIR = Path("/app/data/teehr")

TEMP_DIR = Path(TEEHR_EVALUATION_DIR, "cache")

CROSSWALK_TABLE = Path(TEMP_DIR, "xwalk_table.parquet")
LOCATIONS = Path(TEMP_DIR, "usgs_locations.parquet")
METRICS_CSV_FILEPATH = Path(TEEHR_EVALUATION_DIR, "metrics.csv")

In [ ]:
MOUNTED_DATA_DIR = Path(os.environ.get("NGIAB_OUTPUT_DIR"))
configuration_name = teehr_utils.sanitize_string(MOUNTED_DATA_DIR.name)
print(f"Evaluating NGIAB output in: {MOUNTED_DATA_DIR}")

In [ ]:
run_options = os.environ.get("RUN_OPTIONS")
if run_options:
    run_options = [int(opt) for opt in run_options.split(",")]
    print(f"Run options: {run_options}")

In [ ]:
if 1 in run_options:
    print("Running option 1: Creating the TEEHR Evaluation")
    teehr_utils.create_teehr_evaluation(
        teehr_evaluation_dir=TEEHR_EVALUATION_DIR,
        ngiab_output_dir=NGIAB_OUTPUT_DIR,
        crosswalk_table_filepath=CROSSWALK_TABLE,
        locations_filepath=LOCATIONS,
        temp_dir=TEMP_DIR,
        configuration_name=configuration_name,
    )

In [ ]:
if 2 in run_options:
    print("Running option 2: Calculating performance metrics")
    teehr_utils.calculate_metrics(
        teehr_evaluation_dir=TEEHR_EVALUATION_DIR,
        metrics_csv_filepath=METRICS_CSV_FILEPATH
    )

In [ ]:
metrics_df = pd.read_csv(METRICS_CSV_FILEPATH)
metrics_df

#### Explore

In [ ]:
ev = teehr.Evaluation(dir_path=TEEHR_EVALUATION_DIR)

In [ ]:
gdf = ev.locations.to_geopandas()
gdf.teehr.locations_map()

In [ ]:
ev.primary_timeseries.to_sdf().show()

In [ ]:
ev.secondary_timeseries.to_sdf().show()

In [ ]:
ev.secondary_timeseries.distinct_values("configuration_name")

In [ ]:
ev.joined_timeseries.to_sdf().show()